In [ ]:
import eclabfiles as ecf
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from config import props, filenames

In [ ]:
data = ecf.to_df(f'{filenames.data}/{props.codename}{props.CV_file}_C01.mpr')

In [ ]:
total_nc = int(data['cycle number'].iloc[-1])
fig = plt.figure()
plt.title(f'CV for {props.codename}{props.CV_file}; {total_nc = }')
plt.xlabel('U [V]')
plt.ylabel('J [mA/cm$^2$]')
for i in range(1, total_nc+1):
    cycle = data[data['cycle number'] == i]
    plt.plot(cycle.Ewe, 100*cycle['<I>']/props.el_area, color=plt.cm.viridis(i/total_nc))

In [ ]:
if not os.path.exists(filenames.notes):
    os.makedirs(filenames.notes)
if os.path.exists(f'{filenames.notes}/CV.png'):
    overwrite = (input('Do you want to overwrite the existing CV plot? [y/n]') in ('y', 'Y'))
    if overwrite:
        fig.savefig(f'{filenames.notes}/CV.png')
else:
    fig.savefig(f'{filenames.notes}/CV.png')